# 设定路径

In [1]:
def setProjectPath(projectPath):
    import os
    import sys
    root=os.path.abspath(projectPath)
    print(root)
    sys.path.append(root)
    os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
    import warnings  
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")

projectPath=r'D:\project_meta\NNproject\NNI'
setProjectPath(projectPath)

D:\project_meta\NNproject\NNI


# 实验一
- 

## 数据集

In [42]:
import torch
from project.dataset.SelfDataset import TableControlFullLoadDataset
from project.dataprocess import FolderTree
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler
from torch.utils.data import Dataset
import pandas as pd
from typing import Any


In [142]:
class TableDataset(Dataset):
    def __init__(self,table:pd.DataFrame) -> None:
        super().__init__()
        self.Table=table
    def __len__(self):
        return len(self.Table)
    def __getitem__(self, index) -> Any:
        # torch.tensor(trainDataTable.drop(columns=['ID','KSS','LEVEL']).iloc[1].values)
        x=torch.tensor(self.Table.drop(columns=['ID','KSS','LEVEL']).iloc[index].values).to(torch.float32)
        y=torch.tensor(self.Table['LEVEL'].iloc[index]).long()-1
        return x,y
#路径
data_root=r'D:\dataset\driver_dataset\DROZY\DROZY'
output_root=r"D:\project_meta\NNproject\NNI\output"

data_path_dict=FolderTree.getDataPath(data_root=data_root)
output_path_dic=FolderTree.getOutPath(output_root=output_root)

#表
bandPowerTable=pd.read_csv(r"D:\project_meta\NNproject\NNI\output\bandpoweruniform.csv")
#数据集分割
trainDataTable,testDataTable=train_test_split(bandPowerTable,test_size=0.35,shuffle=True)

#数据加载器
train_dataset=TableDataset(trainDataTable)
train_dataloader=DataLoader(train_dataset,batch_size=512,shuffle=True)

test_dataset=TableDataset(testDataTable)
test_dataloader=DataLoader(test_dataset,batch_size=512)

In [130]:
torch.tensor(trainDataTable['ID'].iloc[1])
torch.tensor(trainDataTable['ID'].iloc[1])

tensor(11)

## MLP模型

In [143]:
import torch.nn as nn
def MLP(numFeature,interWidth,classes):
    interWidth.insert(0,numFeature)
    interWidth.append(classes)
    net=nn.Sequential()
    for i in range(len(interWidth)-2) :
        net.append(nn.Linear(interWidth[i],interWidth[i+1]))
        #net.append(nn.Dropout(0.5))
        net.append(nn.ReLU())
    net.append(nn.Linear(interWidth[-2],interWidth[-1]))
    return net
    pass


## 模型验证

In [144]:
from torchsummary import summary
InterWidth=[32,64,16]
batch=next(iter(train_dataloader))
NumFeature=(batch[0]).shape[-1]
classes=NumFeature
print(f'数据维度：{NumFeature}\n类别数：{classes}')

print("InterWidth",InterWidth)

net=MLP(numFeature=NumFeature,interWidth=InterWidth,classes=classes)
print(net)
summary(net,(1,NumFeature),batch_size=5,device='cpu')

数据维度：25
类别数：25
InterWidth [32, 64, 16]
Sequential(
  (0): Linear(in_features=25, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=16, bias=True)
  (5): ReLU()
  (6): Linear(in_features=16, out_features=25, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [5, 1, 32]             832
              ReLU-2                 [5, 1, 32]               0
            Linear-3                 [5, 1, 64]           2,112
              ReLU-4                 [5, 1, 64]               0
            Linear-5                 [5, 1, 16]           1,040
              ReLU-6                 [5, 1, 16]               0
            Linear-7                 [5, 1, 25]             425
Total params: 4,409
Trainable params: 4,409
Non-trainable params: 0
----------------------------

In [133]:
batch=next(iter(train_dataloader))


In [134]:
batch[0].shape

torch.Size([512, 25])

## 训练模型

In [135]:
from d2l import torch as d2l
import torch.nn as nn 
import torch.nn.functional as F
import torch
import datetime
import os

### 参数权重存储路径

In [136]:

device=torch.device('cuda')
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y_%m_%d_%H_%M_%S")
parameterFolder=os.path.join(r"D:\project_meta\NNproject\NNI\output\model_parameter\FilterSignalCNN1D",f"{formatted_time}")
print(parameterFolder)

D:\project_meta\NNproject\NNI\output\model_parameter\FilterSignalCNN1D\2023_11_13_12_41_27


### 超参数

In [137]:
LR=0.5
Optimizer=torch.optim.SGD
LossFuntion=nn.CrossEntropyLoss
LRScheduler=torch.optim.lr_scheduler.StepLR
StepLRGamma=0.7
StepLRStep=80
LayerSet=[256, 512, 64, 3]
NumFeature=batch[0].shape[-1]
LevelClasses=3
KSSClasses=10
EochsNum=500

In [138]:
net=MLP(NumFeature,LayerSet,LevelClasses)
optim=Optimizer(net.parameters(),lr=LR,)
lrScheduler=LRScheduler(optim,StepLRStep,StepLRGamma)
lossFuntion=LossFuntion()

In [139]:
batch[0].to(torch.device('cpu'))
net.to(torch.device('cpu'))
x=batch[0]
print(x.shape)
y_hat=net(x)
y=batch[1]
lossf=torch.nn.CrossEntropyLoss()


torch.Size([512, 25])


In [140]:
y_hat=torch.squeeze(y_hat, dim=1)
print(y.shape,y_hat.shape)
lossf(y_hat,torch.tensor(y)-1)

torch.Size([512]) torch.Size([512, 3])


C:\Users\Frantic\AppData\Local\Temp\ipykernel_26816\3759476486.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lossf(y_hat,torch.tensor(y)-1)


tensor(1.1202, grad_fn=<NllLossBackward0>)

In [90]:
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y_%m_%d_%H_%M_%S")
parameterFolder=os.path.join(r"D:\project_meta\NNproject\NNI\output\model_parameter\FilterSignalCNN1D",f"{formatted_time}")
print(parameterFolder)
os.makedirs(parameterFolder)

best_train_acc=0
best_train_loss=0
best_test_acc=0
stagnate_times=0
temp_acc=0
num_batches = len(train_dataloader)
device=torch.device('cuda')
net.to(device)
  

D:\project_meta\NNproject\NNI\output\model_parameter\FilterSignalCNN1D\2023_11_13_12_25_18


Sequential(
  (0): Linear(in_features=25, out_features=25, bias=True)
  (1): ReLU()
  (2): Linear(in_features=25, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=512, bias=True)
  (5): ReLU()
  (6): Linear(in_features=512, out_features=64, bias=True)
  (7): ReLU()
  (8): Linear(in_features=64, out_features=3, bias=True)
  (9): ReLU()
  (10): Linear(in_features=3, out_features=3, bias=True)
  (11): ReLU()
  (12): Linear(in_features=3, out_features=3, bias=True)
)

In [145]:
from project.trainer.ClassfierTrainer import ClassfierTrainer
trianer=ClassfierTrainer(net,EochsNum,optim,lossFuntion)
batch=next(iter(train_dataloader))
trianer.TrainBatch(net.to(torch.device('cpu')),batchData=batch,optimizer=optim,lossFunction=lossFuntion)

(tensor([1, 0, 0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 1, 1, 0, 1, 2, 2, 1, 2, 1, 0, 1, 2,
         2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 0, 1, 2, 1, 2, 0, 2, 1, 2, 2, 1, 0, 0, 2,
         2, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 0, 1, 1, 0, 0, 0, 2,
         2, 2, 0, 2, 2, 0, 2, 1, 1, 0, 2, 0, 2, 2, 0, 1, 0, 0, 1, 0, 1, 2, 0, 2,
         1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 2, 2, 0, 0, 2, 0, 1, 0, 0, 1, 1, 0, 1,
         0, 1, 1, 0, 2, 1, 2, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
         0, 1, 1, 2, 0, 0, 2, 0, 0, 0, 2, 1, 2, 2, 0, 2, 2, 1, 0, 0, 2, 1, 0, 2,
         0, 1, 2, 0, 2, 0, 0, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 0, 2, 0, 1,
         1, 2, 2, 1, 0, 1, 1, 1, 1, 2, 1, 0, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1,
         1, 1, 1, 1, 2, 1, 2, 1, 2, 0, 2, 1, 2, 1, 2, 0, 0, 1, 1, 1, 1, 0, 2, 2,
         0, 2, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1,
         2, 2, 2, 0, 2, 2, 1, 0, 1, 0, 1, 0, 1, 1, 1, 2, 0, 2, 1, 1, 1, 2, 1, 2,
         1, 2, 1, 1, 2, 0, 0

# 实验二

In [4]:
from typing import Any
from project.dataset.SelfDataset import TensorTableDataset,TableDataset
from project.dataprocess import FolderTree
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler
import pandas as pd
#路径
data_root=r'D:\dataset\driver_dataset\DROZY\DROZY'
output_root=r"D:\project_meta\NNproject\NNI\output"

data_path_dict=FolderTree.getDataPath(data_root=data_root)
output_path_dic=FolderTree.getOutPath(output_root=output_root)
class BandPowerDataset(TableDataset):
    def __getitem__(self, index) -> Any:
        x,y=super().__getitem__(index)
        return x,y-1
#表
bandPowerTable=pd.read_csv(r"D:\project_meta\NNproject\NNI\output\bandpoweruniform.csv")
#数据集分割
trainDataTable,testDataTable=train_test_split(bandPowerTable,test_size=0.35,shuffle=True)

#数据加载器
train_dataset=BandPowerDataset(trainDataTable,label='LEVEL',drop=['KSS','ID'])
train_dataloader=DataLoader(train_dataset,batch_size=512,shuffle=True)

test_dataset=BandPowerDataset(testDataTable,label='LEVEL',drop=['KSS','ID'])
test_dataloader=DataLoader(test_dataset,batch_size=512)


In [5]:
next(iter(test_dataloader))

[tensor([[0.8617, 0.0863, 0.0094,  ..., 0.0347, 0.0970, 0.1038],
         [0.5527, 0.2558, 0.0298,  ..., 0.1395, 0.2074, 0.1934],
         [0.6498, 0.0562, 0.1105,  ..., 0.4304, 0.3015, 0.0298],
         ...,
         [0.4314, 0.2560, 0.1974,  ..., 0.3525, 0.1686, 0.0985],
         [0.4458, 0.1740, 0.0820,  ..., 0.1834, 0.2510, 0.1910],
         [0.8975, 0.0219, 0.0092,  ..., 0.0623, 0.2797, 0.1872]]),
 tensor([1, 0, 1, 2, 0, 1, 2, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 2, 2, 0, 1, 0, 0, 2,
         1, 1, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1,
         1, 2, 2, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 2, 2, 2, 0, 0, 1, 2, 1,
         1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 0, 2, 1, 2,
         1, 1, 2, 0, 0, 2, 1, 0, 1, 1, 0, 1, 0, 1, 2, 0, 1, 1, 0, 2, 2, 2, 0, 0,
         0, 2, 2, 2, 2, 2, 0, 1, 0, 1, 0, 2, 2, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 2,
         1, 1, 2, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2,
         1, 2, 2, 2, 2, 2, 